In [4]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from models.kernel_predictor import MultiKernelPredictor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import pickle
import os

# GPU 설정
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("\033[92mGPU Detected. Configuring GPU...\033[0m")
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)  # 메모리 동적 할당
else:
    print("\033[91mNo GPU detected. Running on CPU.\033[0m")

GPU Detected. Configuring GPU...


In [ ]:
def create_first_sales_column_dict(df, save_result=False):
    # sell_prices의 각 행에서 NaN이 끝나고 처음으로 숫자인 column을 찾기
    first_sales_column_dict = {}
    
    for idx in df.index:
        row = df.iloc[idx]
        # state_id와 item_id로 키 튜플 생성
        key = (row['state_id'], row['item_id'])
        
        # NaN 값들의 위치를 찾습니다
        nan_mask = row.isna()
        
        if not nan_mask.any():
            # NaN이 없는 경우 첫 번째 데이터 컬럼('d_1')을 저장
            first_sales_column_dict[key] = 'd_1'
        else:
            # 마지막 NaN의 위치를 찾습니다
            last_nan_idx = nan_mask[::-1].idxmax()
            # 마지막 NaN의 위치 이후의 첫 번째 숫자가 있는 컬럼을 찾습니다
            last_nan_position = row.index.get_loc(last_nan_idx)
            first_number_col = row.index[last_nan_position + 1]
            first_sales_column_dict[key] = first_number_col
    
    if save_result:
        with open('../data/preprocessed/first_sales_column_dict.pkl', 'wb') as f:
            pickle.dump(first_sales_column_dict, f)
            
    return first_sales_column_dict

# 판매량 데이터 (아이템별 상이)
sales = pd.read_csv("../data/preprocessed/sales.csv")
# 판매 가격 데이터 (아이템별 상이)
sell_prices = pd.read_csv("../data/preprocessed/sell_prices.csv")
# 주말 여부 데이터 (1941일)
is_weekend = pd.read_csv("../data/preprocessed/is_weekend.csv")
# 이벤트 데이터 (1941일)
event_type_cultural = pd.read_csv("../data/preprocessed/event_type_cultural.csv")
event_type_national = pd.read_csv("../data/preprocessed/event_type_national.csv")
event_type_religious = pd.read_csv("../data/preprocessed/event_type_religious.csv")
event_type_sporting = pd.read_csv("../data/preprocessed/event_type_sporting.csv")

# 판매 시작 시점 
# first_sales_column_dict = create_first_sales_column_dict(sell_prices, save_result==True)
with open('../data/preprocessed/first_sales_column_dict.pkl', 'rb') as f:
    first_sales_column_dict = pickle.load(f)


In [8]:
def align_by_first_sale_day(state_item_id, first_sales_day):
    # 주 ID, 아이템 ID
    state_id, item_id = state_item_id

    # 필요한 데이터 필터링 (한 번씩만 필터링)
    sales_df = sales.query("state_id == @state_id and item_id == @item_id").iloc[0]
    price_df = sell_prices.query("state_id == @state_id and item_id == @item_id").iloc[0]
    
    # 각 데이터의 슬라이싱 시작 인덱스 계산
    start_col_sales = sales_df.index.get_loc(first_sales_day)
    start_col_price = price_df.index.get_loc(first_sales_day)

    # 각 데이터 슬라이싱
    sales_data = sales_df.iloc[start_col_sales:].values
    price_data = price_df.iloc[start_col_price:].values
    
    # 고정된 데이터 (weekend와 이벤트 데이터)
    start_col_weekend_event = is_weekend.columns.get_loc(first_sales_day)
    weekend_data = is_weekend.iloc[0, start_col_weekend_event:].values
    cultural_data = event_type_cultural.iloc[0, start_col_weekend_event:].values
    national_data = event_type_national.iloc[0, start_col_weekend_event:].values
    religious_data = event_type_religious.iloc[0, start_col_weekend_event:].values
    sporting_data = event_type_sporting.iloc[0, start_col_weekend_event:].values
    
    # 출력
    return {
        "sales": sales_data,
        "price": price_data,
        "weekend": weekend_data,
        "cultural": cultural_data,
        "national": national_data,
        "religious": religious_data,
        "sporting": sporting_data,
    }

In [5]:
def create_generator_for_each_item(num_kernels, look_back_window_size, look_forward_window_size):
    for state_item_id, first_sales_day in first_sales_column_dict.items(): # 아이템별로
        data = sort_by_first_sale_day(state_item_id, first_sales_day)

        kernel_sizes = [k for k in fourier_results[state_item_id]['selected_periods'][:num_kernels]]
        max_kernel_size = max(kernel_sizes)
        look_back_window_size = max(look_back_window_size, max_kernel_size)

        # 슬라이딩 윈도우 데이터 생성
        input_data = []
        output_data = []
        for i in range(len(data["sales"]) - look_back_window_size - look_forward_window_size): # 슬라이딩 윈도우
            ##### 슬라이딩 윈도우 간격
            input = {
                "sales": data["sales"][i:i+look_back_window_size],
                "price": data["price"][i:i+look_back_window_size],
                "weekend": data["weekend"][i:i+look_back_window_size],
                "cultural": data["cultural"][i:i+look_back_window_size],
                "national": data["national"][i:i+look_back_window_size],
                "religious": data["religious"][i:i+look_back_window_size],
                "sporting": data["sporting"][i:i+look_back_window_size],
            }
            output = data["sales"][i+look_back_window_size:i+look_back_window_size+look_forward_window_size]
            input_data.append(input)
            output_data.append(output)

        yield input_data, output_data, kernel_sizes  # 아이템별로 훈련/테스트 데이터 반환

In [45]:
def extract_periods_in_sales_data_with_fourier(sales_data, low_cutoff_period, high_cutoff_period, 
                                               save_dominant_period_and_normalized_magnitude, 
                                               save_plot, snr_percentile, save_final_periods, save_path):
    """
    아이템별 판매 데이터를 분석하여 주기 분석과 SNR 계산을 수행하는 함수.
    
    Args:
    - sales_data: 판매 데이터 (시간 순서대로)
    - low_cutoff_period: 저주파/긴 주기 제거 (기본값 365일)
    - high_cutoff_period: 고주파/짧은 주기 제거 (기본값 3일)
    - save_dominant_period_and_normalized_magnitude: dominant_period_and_normalized_magnitude 엑셀 파일 저장 여부
    - save_plot: 그래프 저장 여부
    - snr_percentile: SNR의 퍼센트로 선택할 주기의 비율 (기본값 0.75)
    - save_final_periods: 최종 주기 결과를 피클로 저장 여부
    - save_path: 저장 경로
    
    Returns:
    - final_periods: 최종적으로 선택된 주기들
    """

    # 폴더가 없으면 생성
    os.makedirs(save_path, exist_ok=True) 

    # 샘플링 주파수와 나이퀴스트 주파수 설정
    sampling_frequency = 1  # 1일 단위 샘플링
    nyquist_frequency = sampling_frequency / 2  # 나이퀴스트 주파수
    
    # 필터링 주파수 설정
    low_cutoff_frequency = 1 / low_cutoff_period  # 저주파/긴 주기 제거 # 주기 ?일 이상 제거
    high_cutoff_frequency = 1 / high_cutoff_period  # 고주파/짧은 주기 제거 # 주기 ?일 이하 제거
    normal_low_cutoff_frequency = low_cutoff_frequency / nyquist_frequency  # 정규화된 컷오프 주파수
    normal_high_cutoff_frequency = high_cutoff_frequency / nyquist_frequency  # 정규화된 컷오프 주파수
    
    # 밴드패스 필터 설계
    b, a = butter(N=3, Wn=[normal_low_cutoff_frequency, normal_high_cutoff_frequency], btype='band', analog=False)
    sales_data_filtered = filtfilt(b, a, sales_data)  # 필터 적용

    # 푸리에 변환
    fft = np.fft.fft(sales_data_filtered)
    frequencies = np.fft.fftfreq(len(sales_data_filtered), d=1)[:len(sales_data_filtered)//2]  # 대칭 구조이기에 절반만 사용
    magnitudes = np.abs(fft)[:len(sales_data_filtered)//2]  # 대칭 구조이기에 절반만 사용
    
    # 나이퀴스트 조건 고려 (에일리어싱 방지) & 주파수 0 제거
    valid_indices = (frequencies <= nyquist_frequency) & (frequencies > 0)
    frequencies = frequencies[valid_indices]
    magnitudes = magnitudes[valid_indices]

    # 진폭순으로 정렬
    sorted_indices = np.argsort(magnitudes)[::-1]
    frequencies = frequencies[sorted_indices]
    magnitudes = magnitudes[sorted_indices]

    # 진폭 스펙트럼 시각화
    if save_plot:
        plt.figure(figsize=(12, 6))

        # 원본 진폭 스펙트럼
        plt.plot(frequencies, magnitudes_original, label='Original Magnitude Spectrum', color='orange')

        # 필터링된 진폭 스펙트럼
        plt.plot(frequencies, magnitudes, label='Filtered Magnitude Spectrum', color='blue')

        # 그래프 설정
        plt.title("Original and Filtered Magnitude Spectrum")
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude')
        plt.legend()
        plt.grid(True)

        # 플롯 저장
        plt.tight_layout()
        plt.savefig(os.path.join(save_path, "amplitude_spectrum_overlap.png"))
        plt.close()
    
    # 주기 계산
    periods = [round(1 / frequency) for frequency in frequencies]  # 주기 반올림    

    # 가장 큰 진폭을 가진 주기와 정규화된 진폭의 퍼센트 데이터프레임에 저장
    dominant_period_and_normalized_magnitude = pd.DataFrame(columns=["Dominant Period", "Normalized Magnitude"])
    dominant_period_and_normalized_magnitude.loc["dominant_period"] = [periods[0], (magnitudes[0] / magnitudes.sum()) * 100] # 백분율

    # 선택된 주기 개수별 SNR 계산
    snr_results = []  # 주기 개수별 SNR 결과
    for i in range(1, len(periods) + 1):
        # 선택된 주기와 진폭
        selected_periods = periods[:i]
        selected_magnitudes = magnitudes[:i]

        # 중복된 주기 제거
        deduplicated_periods = []  # 중복 제거된 주기
        deduplicated_magnitudes = []  # 중복 제거된 진폭
        deduplicated_indices = []  # 선택된 주파수 인덱스
        for j, selected_period in enumerate(selected_periods):  # 선택된 주기 반복
            if selected_period not in deduplicated_periods:  # (이미 진폭순으로 내림차순이기 때문에) 선택된 주기가 중복되지 않을 때만 추가
                deduplicated_periods.append(selected_period)
                deduplicated_magnitudes.append(selected_magnitudes[j])
                deduplicated_indices.append(sorted_indices[j])

        # 인버스 푸리에 변환
        inverse_fft = np.zeros_like(fft, dtype=complex)
        for selected_index in deduplicated_indices:
            inverse_fft[selected_index] = fft[selected_index]
            inverse_fft[-selected_index] = fft[-selected_index]
        reconstructed_sales_data = np.fft.ifft(inverse_fft).real

        # SNR 계산
        original_energy = np.sum(np.abs(sales_data_filtered) ** 2)
        error_energy = np.sum(np.abs(sales_data_filtered - reconstructed_sales_data) ** 2)
        current_snr = 10 * np.log10(original_energy / error_energy)
        snr_results.append(current_snr)
    
    # 가장 큰 진폭을 가진 주기와 정규화된 진폭의 퍼센트 엑셀에 저장
    if save_dominant_period_and_normalized_magnitude:
        dominant_period_and_normalized_magnitude.to_excel(f"../data/fourier/dominant_period_and_normalized_magnitude_low{low_cutoff_period}_high{high_cutoff_period}_snr{snr_percentile}.xlsx")

    # 시각화
    if save_plot:
        plt.figure(figsize=(15, 15))

        # 원본 판매 데이터와 복원 판매 데이터 시각화
        plt.subplot(211)
        plt.plot(sales_data_filtered, label='Original Sales Data')
        plt.plot(reconstructed_sales_data, label='Reconstructed Sales Data')
        plt.xlabel("Time Steps")
        plt.ylabel("Sales")

        # 누적 SNR 시각화
        plt.subplot(212)
        plt.plot(range(1, len(snr_results) + 1), snr_results)
        plt.xlabel('Number of Periods')
        plt.ylabel('SNR (dB)')

        plt.legend()
        plt.tight_layout()
        plt.savefig("../data/fourier/sales_data_plot.png")
        plt.close()

    # SNR 결과 정규화
    max_snr, min_snr = max(snr_results), min(snr_results)
    normalized_snr = [(x - min_snr) / (max_snr - min_snr) for x in snr_results]  # 최소-최대 정규화

    # SNR의 퍼센트에 해당하는 인덱스 찾기
    percentile_snr = np.percentile(normalized_snr, snr_percentile * 100)  # ?%에 해당하는 SNR 값
    percentile_snr_index = (np.abs(np.array(normalized_snr) - percentile_snr)).argmin()  # 가장 가까운 값의 인덱스

    # 선택된 SNR의 퍼센트에 해당되는 주기 개수만큼 가져오기
    final_periods = periods[:percentile_snr_index + 1]

    # final_periods 피클로 저장
    if save_final_periods:
        
        final_periods_file_name = f"final_periods_low{low_cutoff_period}_high{high_cutoff_period}_snr{snr_percentile}.pkl"
        result = {
            "final_periods": final_periods
        }
        with open(os.path.join(save_path, final_periods_file_name), 'wb') as f:
            pickle.dump(result, f)

    return final_periods


In [50]:
from scipy.signal import butter, filtfilt, freqz
from sklearn.preprocessing import QuantileTransformer

# 파라미터 설정
test_size = 0.2

n_epochs = 300
initial_learning_rate = 5e-4
lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate,
    first_decay_steps=1000,
    t_mul=2.0,
    m_mul=0.9,
    alpha=1e-5
)

batch_size = 64

num_kernels = 20 # 상위 몇 개 커널 사용할지
look_back_window_size = 1
look_forward_window_size = 1

num_filters=128

dominant_period_and_normalized_magnitude = pd.DataFrame(columns=['dominant_period', 'normalized_magnitude'], index=first_sales_column_dict.keys())  # 가장 큰 진폭을 가진 주파수 정규화된 진폭의 퍼센트 저장

for idx, (state_item_id, first_sales_day) in enumerate(first_sales_column_dict.items()): # 아이템별로
    aligned_data = align_by_first_sale_day(state_item_id, first_sales_day)

    ### 판매 데이터 
    sales_data = aligned_data["sales"]
    
    ## 푸리에 변환을 통해 주기 추출
    sampling_frequency = 1 # 1일 단위 샘플링
    nyquist_frequency = sampling_frequency/2 # 나이퀴스트 주파수
    
    # 밴드패스 필터 적용 (저주파/고주파 제거)
    low_cutoff_frequency = 1/365 # 저주파/긴 주기(트렌드) 제거 # 주기 ?일 이상 제거
    high_cutoff_frequency = 1/3 # 고주파/짧은 주기(노이즈) 제거 # 주기 ?일 이하 제거
    normal_low_cutoff_frequency = low_cutoff_frequency / nyquist_frequency  # 정규화된 컷오프 주파수
    normal_high_cutoff_frequency = high_cutoff_frequency / nyquist_frequency  # 정규화된 컷오프 주파수
    b, a = butter(N=3, Wn=[normal_low_cutoff_frequency, normal_high_cutoff_frequency], btype='band', analog=False)
    sales_data = filtfilt(b, a, sales_data)  # 필터 적용

    # 푸리에 변환
    fft = np.fft.fft(sales_data)
    frequencies = np.fft.fftfreq(len(sales_data), d=1)[:len(sales_data)//2]  # 대칭 구조이기에 절반만 사용
    magnitudes = np.abs(fft)[:len(sales_data)//2] # 대칭 구조이기에 절반만 사용
    
    # 나이퀴스트 조건 고려 (에일리어싱 방지) & 주파수 0 제거
    valid_indices = (frequencies <= nyquist_frequency) & (frequencies > 0)
    frequencies = frequencies[valid_indices]
    magnitudes = magnitudes[valid_indices]

    # 진폭순으로 정렬
    sorted_indices = np.argsort(magnitudes)[::-1]
    frequencies = frequencies[sorted_indices]
    magnitudes = magnitudes[sorted_indices]

    # 주기 계산
    periods = [round(1/frequency) for frequency in frequencies]  # 주기 반올림	

    # 가장 큰 진폭을 가진 주기와 정규화된 진폭의 퍼센트를 데이터프레임에 저장
    dominant_period_and_normalized_magnitude.loc[state_item_id] = [periods[0], (magnitudes[0]/magnitudes.sum())*100]

    # 선택된 주기 개수별 SNR 계산
    snr_results = [] # 주기 개수별 SNR 결과
    for i in range(1, len(periods) + 1):
        # 선택된 주기와 진폭
        selected_periods = periods[:i]
        selected_magnitudes = magnitudes[:i]

        # 중복된 주기 제거
        deduplicated_periods = []  # 중복 제거된 주기
        deduplicated_magnitudes = []  # 중복 제거된 진폭
        deduplicated_indices = []  # 선택된 주파수 인덱스
        for j, selected_period in enumerate(selected_periods): # 선택된 주기 반복
            if selected_period not in deduplicated_periods: # (이미 진폭순으로 내림차순이기 때문에) 선택된 주기가 중복되지 않을 때만 추가
                deduplicated_periods.append(selected_period)
                deduplicated_magnitudes.append(selected_magnitudes[j])
                deduplicated_indices.append(sorted_indices[j])

        # 인버스 푸리에 변환
        inverse_fft = np.zeros_like(fft, dtype=complex)
        for selected_index in deduplicated_indices:
            inverse_fft[selected_index] = fft[selected_index] 
            inverse_fft[-selected_index] = fft[-selected_index]
        reconstructed_sales_data = np.fft.ifft(inverse_fft).real

        # SNR 계산
        original_energy = np.sum(np.abs(sales_data) ** 2)
        error_energy = np.sum(np.abs(sales_data - reconstructed_sales_data) ** 2)
        current_snr = 10 * np.log10(original_energy / error_energy)
        snr_results.append(current_snr)
    
    # 가장 큰 진폭을 가진 주기와 정규화된 진폭의 퍼센트 엑셀에 저장
    dominant_period_and_normalized_magnitude.to_excel('../data/fourier/dominant_period_and_normalized_magnitude.xlsx')

    # 원본 판매 데이터와 복원 판매 데이터 시각화
    plt.figure(figsize=(15, 15))

    plt.subplot(211)
    plt.plot(sales_data, label='Original Sales Data')
    plt.plot(reconstructed_sales_data, label='Reconstructed Sales Data')
    plt.xlabel("Time Steps")
    plt.ylabel("Sales")

    # 누적 SNR 시각화
    plt.subplot(212)
    plt.plot(range(1, len(snr_results) + 1), snr_results)
    plt.xlabel('Number of Periods')
    plt.ylabel('SNR (dB)')

    plt.legend()
    plt.tight_layout()
    plt.show() # plt.savefig(f"../data/fourier/plot/{state_item_id}.png")
    plt.close()

    # SNR 결과 정규화
    max_snr, min_snr = max(snr_results), min(snr_results)
    normalized_snr = [(x-min_snr)/(max_snr-min_snr) for x in snr_results] # 최소-최대 정규화

    # SNR의 퍼센트에 해당하는 인덱스 찾기
    snr_percentile = 0.75  # ?% # 입력
    percentile_snr = np.percentile(normalized_snr, snr_percentile*100) # ?%에 해당하는 SNR 값
    percentile_snr_index = (np.abs(np.array(normalized_snr) - percentile_snr)).argmin()  # 가장 가까운 값의 인덱스

    # 선택된 SNR의 퍼센트에 해당되는 주기 개수만큼 가져오기
    final_periods = periods[:percentile_snr_index+1]

    break


    for item_idx, (input_data, output_data, kernel_sizes, kernel_strengths) in enumerate(item_windows_generator): # 윈도우별로 (배치별로)
        print(f"\033[93mItem {item_idx}: {state_item_id}\033[0m")

        print(f"\033[92mPreparing Data\033[0m")
        split_index = int(len(input_data) * (1 - test_size))
        train_data = [(input_data[i], output_data[i]) for i in range(split_index)]
        test_data = [(input_data[i], output_data[i]) for i in range(split_index - look_back_window_size, len(input_data))]
        
        x_scaler = QuantileTransformer(output_distribution='normal')
        y_scaler = QuantileTransformer(output_distribution='normal')

        X_sales_train = np.array([x['sales'] for x, _ in train_data]).astype(np.float32)  # (train_size, look_back_window_size)
        X_sales_train = x_scaler.fit_transform(X_sales_train)	

        
        y_train = np.array([y for _, y in train_data]).astype(np.float32)  # (train_size, look_forward_window_size)
        y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1, look_forward_window_size)

        X_sales_test = np.array([x['sales'] for x, _ in test_data]).astype(np.float32)  # (test_size, look_back_window_size)
        X_sales_test = x_scaler.transform(X_sales_test)


        y_test = np.array([y for _, y in test_data]).astype(np.float32)  # (test_size, look_forward_window_size)
        y_test = y_scaler.transform(y_test.reshape(-1, 1)).reshape(-1, look_forward_window_size)

        print(f"\033[92mPreparing Loss and Optimizer\033[0m")
        loss = tf.keras.losses.Huber(delta=1.0)
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr_schedule,
            clipnorm=0.5
        )

        print(f"\033[92mBuilding Model\033[0m")
        model = MultiKernelPredictor(
            kernel_sizes=kernel_sizes,
        )

        print(f"\033[92mTraining\033[0m")
        train_dataset = tf.data.Dataset.from_tensor_slices(({
                                                            "sales": X_sales_train, 
                                                            }, y_train))
        train_dataset = train_dataset.batch(batch_size)
        
        model.compile(
            optimizer=optimizer,
            loss=custom_peak_loss
        )
        
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=20,
            restore_best_weights=True
        )
        lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
                  
        history = model.fit(
            train_dataset,
            epochs=n_epochs,
    		callbacks=[early_stopping]
        )

        print(f"\033[92mTesting\033[0m")
        test_dataset = tf.data.Dataset.from_tensor_slices(({
                                                            "sales": X_sales_test, 
                                                            "price": X_price_test,
                                                            "weekend": X_weekend_test, 
                                                            "cultural": X_cultural_test,
                                                            "national": X_national_test, 
                                                            "religious": X_religious_test,
                                                            "sporting": X_sporting_test, 
                                                            }, y_test))
        test_dataset = test_dataset.batch(batch_size)
        
        test_loss = model.evaluate(test_dataset, verbose=1)
        print(f"Test Loss: {test_loss}")

        # 예측과 실제값 시각화 준비
        predictions = model.predict(test_dataset)

        # 스케일링 역변환
        labels = y_scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(-1)
        predictions = y_scaler.inverse_transform(predictions.reshape(-1, 1)).reshape(-1)

        # 그래프 그리기
        plt.figure(figsize=(12, 6))
        plt.plot(labels, label='Labels', linewidth=2)
        plt.plot(predictions, label='Predictions', linewidth=2, linestyle='--')
        plt.xlabel("Time Steps")
        plt.ylabel("Values")
        plt.legend()

        # 그림 저장
        plt.savefig(f"Item {item_idx}: {state_item_id}.png")
        plt.show()

        break
    break
    #     # print(f"\033[92mAnalyzing\033[0m")
    #     # # 모델 해석 단계
    #     # analyzer = AnalyzeModel(model, X_sales_test, X_aux_test)
    #     # analyzer.pdp(feature_index=0)  # 예시: 첫 번째 보조 특성에 대한 PDP
    #     # analyzer.ice(feature_index=0)  # 예시: 첫 번째 보조 특성에 대한 ICE
    #     # analyzer.feature_weights()  # 모델의 학습된 가중치 출력

    # # # 모델 저장
    # # model.save(f"model_{state_item_id}.h5")

    # # # 모델 가중치 초기화
    # # model.reset_states()

    # break

('CA', 'FOODS_1_001')
